In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.stats import gmean

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
import gc

np.random.seed(42069)
warnings.simplefilter('ignore')
%matplotlib inline

In [2]:

params = {
    'figure.figsize': (10, 5)
     }
plt.rcParams.update(params)

In [6]:

# This will download data from yahoo finance - more than enough for prototyping
df = yf.download(
        # tickers list or string as well
        tickers = "^tnx",
        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "5y",
        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1d",
        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',
        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,
        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = True,
        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,
        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )

[*********************100%***********************]  1 of 1 completed


In [4]:
df.shape

(5, 5)

In [7]:
print(df)

             Open   High    Low  Close  Volume
Date                                          
2017-02-13  2.436  2.452  2.427  2.434       0
2017-02-14  2.441  2.502  2.431  2.470       0
2017-02-15  2.484  2.524  2.483  2.502       0
2017-02-16  2.475  2.492  2.438  2.450       0
2017-02-17  2.406  2.429  2.400  2.425       0
...           ...    ...    ...    ...     ...
2022-02-07  1.921  1.939  1.910  1.916       0
2022-02-08  1.943  1.971  1.943  1.954       0
2022-02-09  1.936  1.947  1.909  1.929       0
2022-02-10  1.929  2.050  1.927  2.031       0
2022-02-11  2.007  2.063  1.934  1.955       0

[1260 rows x 5 columns]


In [47]:
def count_annotation(Data, name, onwhat, what_bull, what_bear, td, window = 50):
    
    Plottable = Data[-window:, ]
    
    fig, ax = plt.subplots(figsize = (10, 5))
    ax.grid()
    
    ax.plot(Plottable[:, onwhat], color = 'black', linewidth = 1.5, label = name)
    
    for i in range(len(Plottable)):
        
        if Plottable[i, what_bull] < 0 and Plottable[i, what_bull] != -td:
            
            x = i
            y = Plottable[i, onwhat]
        
            ax.annotate(int(Plottable[i, what_bull]), xy = (x, y), textcoords = "offset points", xytext = (0, - 10), ha = 'center',
                        color = 'blue')
            
        elif Plottable[i, what_bull] == -td:
            
            x = i
            y = Plottable[i, onwhat]
        
            ax.annotate(int(Plottable[i, what_bull]), xy = (x, y), textcoords = "offset points", xytext = (0, - 10), ha = 'center',
                        color = 'red')
            
        elif Plottable[i, what_bear] > 0 and Plottable[i, what_bear] != td:
            
            x = i
            y = Plottable[i, onwhat]
        
            ax.annotate(int(Plottable[i, what_bear]), xy = (x, y), textcoords = "offset points", xytext = (0, 10), ha = 'center',
                        color = 'blue' )

        elif Plottable[i, what_bear] == td:
            
            x = i
            y = Plottable[i, onwhat]
        
            ax.annotate(int(Plottable[i, what_bear]), xy = (x, y), textcoords = "offset points", xytext = (0, 10), ha = 'center',
                        color = 'red' )
                     
    ax.set_facecolor((0.95, 0.95, 0.95)) 
    plt.legend()
    
def adder(Data, times):
    
    for i in range(1, times + 1):
    
        new = np.zeros((len(Data), 1), dtype = float)
        Data = np.append(Data, new, axis = 1)

    return Data

def deleter(Data, index, times):
    
    for i in range(1, times + 1):
    
        Data = np.delete(Data, index, axis = 1)

    return Data
   
def jump(Data, jump):
    
    Data = Data[jump:, ]
    
    return Data

def rounding(Data, how_far):
    
    Data = Data.round(decimals = how_far)
    
    return Data
            







def rolling_correlation(Data, first_data, second_data, lookback, where):
    
    # Adding an extra column
    Data = adder(Data, 1)
    
    for i in range(len(Data)):
        
        try:
            Data[i, where] = pearsonr(Data[i - lookback + 1:i + 1, first_data], Data[i - lookback + 1:i + 1, second_data])[0]
            
             
        except ValueError:
            pass
    
    Data = jump(Data, lookback) 
    
    return Data

def auto_correlation(Data, first_data, second_data, shift_degree, lookback, where):
    
    new_array = shift(Data[:, first_data], shift_degree, cval = 0)
    new_array = np.reshape(new_array, (-1, 1))
    
    Data = np.concatenate((Data, new_array), axis = 1)
    Data = adder(Data, 1)
    
    for i in range(len(Data)):
        
        try:
            Data[i, where] = pearsonr(Data[i - lookback + 1:i + 1, first_data], Data[i - lookback + 1:i + 1, second_data])[0]
            
            
        except ValueError:
            pass
    
    Data = jump(Data, lookback) 
    Data = deleter(Data, where - 1, 1)
    
    return Data

def volatility(Data, lookback, what, where):
    
    # Adding an extra column
    Data = adder(Data, 1)
    
    for i in range(len(Data)):
        
        try:
            Data[i, where] = (Data[i - lookback + 1:i + 1, what].std())
    
        except IndexError:
            pass
     
    # Cleaning
    Data = jump(Data, lookback)    
     
    return Data


def ma(Data, lookback, close, where): 
    
    Data = adder(Data, 1)
    
    for i in range(len(Data)):
           
            try:
                Data[i, where] = (Data[i - lookback + 1:i + 1, close].mean())
            
            except IndexError:
                pass
            
    # Cleaning
    Data = jump(Data, lookback)
    
    return Data

def ema(Data, alpha, lookback, what, where):
    
    alpha = alpha / (lookback + 1.0)
    beta  = 1 - alpha
    
    # First value is a simple SMA
    Data = ma(Data, lookback, what, where)
    
    # Calculating first EMA
    Data[lookback + 1, where] = (Data[lookback + 1, what] * alpha) + (Data[lookback, where] * beta)

    # Calculating the rest of EMA
    for i in range(lookback + 2, len(Data)):
            try:
                Data[i, where] = (Data[i, what] * alpha) + (Data[i - 1, where] * beta)
        
            except IndexError:
                pass
            
    return Data 


def atr(Data, lookback, high, low, close, where, genre = 'Smoothed'):
    
    # Adding the required columns
    Data = adder(Data, 1)
    
    # True Range Calculation
    for i in range(len(Data)):
        
        try:
            
            Data[i, where] =   max(Data[i, high] - Data[i, low],
                               abs(Data[i, high] - Data[i - 1, close]),
                               abs(Data[i, low] - Data[i - 1, close]))
            
        except ValueError:
            pass
        
    Data[0, where] = 0   
    
    if genre == 'Smoothed':
        
        # Average True Range Calculation
        Data = ema(Data, 2, lookback, where, where + 1)
    
    if genre == 'Simple':
    
        # Average True Range Calculation
        Data = ma(Data, lookback, where, where + 1)
    
    # Cleaning
    Data = deleter(Data, where, 1)
    Data = jump(Data, lookback)

    return Data

def Synatr(Data,  high, low, close, where, genre = 'Smoothed'):
    
    # Adding the required columns
    Data = adder(Data, 1)
    
    lookback =2 
    
    # True Range Calculation
    for i in range(len(Data)):
        
        try:
            
            Data[i, where] =   max(Data[i, high] - Data[i, low],
                               abs(Data[i, high] - Data[i - 1, close]),
                               abs(Data[i, low] - Data[i - 1, close]))
            
        except ValueError:
            pass
        
    Data[0, where] = 0   
    
    if genre == 'Smoothed':
        
        # Average True Range Calculation
        Data = ema(Data, 2, lookback, where, where + 1)
    
    if genre == 'Simple':
    
        # Average True Range Calculation
        Data = np.divide(ma(Data, 2, where, where + 1),ma(Data, 1, where, where + 1))
    
    # Cleaning
    Data = deleter(Data, where, 1)
    Data = jump(Data, lookback)

    return Data


np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [40]:
my_data=atr(df,2,1,2,3,5,'Simple')

In [41]:
my_data=atr(my_data,1,1,2,3,6,'Simple')

In [48]:
my_data=Synatr(my_data,1,2,3,7,'Simple')

ValueError: operands could not be broadcast together with shapes (1251,9) (1252,9) 

In [16]:
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2016-10-18,195.615868,195.679776,194.730190,195.131943,76869700
2016-10-19,195.414981,195.981080,195.031494,195.652374,66519200
2016-10-20,195.277985,195.880614,194.584058,195.287125,73639800
2016-10-21,195.287150,195.469761,194.264503,195.378448,89089100
2016-10-24,196.309773,196.601962,195.834974,196.209335,60146600


In [33]:
print(my_data)

[[194.675392 195.780197 194.419720 ... 75705500.000000 1.136773 1.360477]
 [195.926316 195.962833 194.556713 ... 77220200.000000 1.383299 1.406120]
 [194.611461 195.332779 193.305779 ... 140623200.000000 1.716560 2.027000]
 ...
 [439.079987 442.660004 438.579987 ... 70236800.000000 5.994995 7.480011]
 [444.750000 446.260010 444.089996 ... 66226800.000000 5.620010 3.760010]
 [443.970001 447.019989 443.269989 ... 33588065.000000 3.755005 3.750000]]


In [17]:
nullvaluecheck = pd.DataFrame(my_data.isna().sum().sort_values(ascending=False)*100/my_data.shape[0],columns=['missing %']).head(60)
nullvaluecheck.style.background_gradient(cmap='PuBu')

AttributeError: 'numpy.ndarray' object has no attribute 'isna'